In [1]:
import json
from pprint import pprint
import sys
import unicodedata

import fiona
from shapely.geometry import shape, mapping
from shapely.ops import cascaded_union
from django.contrib.gis.geos import GEOSGeometry
from fiona.crs import to_string, from_epsg
from fiona.transform import transform_geom
from pyproj import Proj

from madmex.models import Region, Country
from madmex.util.spatial import feature_transform

In [2]:
#name_admin = 'MEX' or column_name = 'AGUACATE_DIANA'
name_admin = 'AGUACATE_DIANA'
#column_name = 'NAME_1' or column_name = ''
column_name = ''
#shapefile = '/LUSTRE/MADMEX/tasks/2019_tasks/MEX_adm_shp/MEX_adm1.shp' 
#or shapefile = '/LUSTRE/MADMEX/tasks/2019_tasks/diana_poligonos_aguacate/poly_inventarios_buff30m/Inventario_1995_buff30m_poly.shp'
shapefile = '/LUSTRE/MADMEX/tasks/2019_tasks/diana_poligonos_aguacate/poly_inventarios_buff30m/Inventario_1995_buff30m_poly.shp'

In [3]:
with fiona.open(shapefile,
                encoding='utf-8') as src:
    fc = list(src)
    src_crs = src.crs
    to_string_crs = to_string(src_crs)
    proj_crs = Proj(src.crs)
    if not proj_crs.is_latlong():
        fc_proj = [feature_transform(x,"+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs",to_string_crs) for x in fc]
    else:
        fc_proj = fc

In [4]:
[feat['geometry']['type'] for feat in fc_proj][0:4]

['Polygon', 'Polygon', 'Polygon', 'Polygon']

In [5]:
shape_list = [shape(feat['geometry']) for feat in fc_proj]

In [6]:
country_shape = cascaded_union(shape_list)

In [7]:
country, _ = Country.objects.get_or_create(the_geom=GEOSGeometry(country_shape.wkt),
                                       name=name_admin)

In [8]:
# Ingest features
for k in range(0,len(fc_proj)):
    shape = shape_list[k]
    geom = GEOSGeometry(shape.wkt,4326)
    
    if not column_name: #needs to generate name for every entry
        name = name_admin + '_%s'
        name_feature = name % fc_proj[k]['id']
    else:
        name_feature = fc_proj[k]['properties'][column_name].replace(" ", "_")
        name_feature_normalized = unicodedata.normalize('NFKD', name_feature).encode('ASCII', 'ignore').decode('utf-8')
        name_feature = name_admin + '_' + name_feature_normalized
    _ = Region.objects.get_or_create(name=name_feature,
                                     the_geom=geom,
                                     country=country)
    print(name_feature)

AGUACATE_DIANA_0
AGUACATE_DIANA_1
AGUACATE_DIANA_2
AGUACATE_DIANA_3
AGUACATE_DIANA_4
AGUACATE_DIANA_5
AGUACATE_DIANA_6
AGUACATE_DIANA_7
AGUACATE_DIANA_8
AGUACATE_DIANA_9
AGUACATE_DIANA_10
AGUACATE_DIANA_11
AGUACATE_DIANA_12
AGUACATE_DIANA_13
AGUACATE_DIANA_14
AGUACATE_DIANA_15
AGUACATE_DIANA_16
AGUACATE_DIANA_17
AGUACATE_DIANA_18
AGUACATE_DIANA_19
AGUACATE_DIANA_20
AGUACATE_DIANA_21
AGUACATE_DIANA_22
AGUACATE_DIANA_23
AGUACATE_DIANA_24
AGUACATE_DIANA_25
AGUACATE_DIANA_26
AGUACATE_DIANA_27
AGUACATE_DIANA_28
AGUACATE_DIANA_29
AGUACATE_DIANA_30
AGUACATE_DIANA_31
AGUACATE_DIANA_32
AGUACATE_DIANA_33
AGUACATE_DIANA_34
AGUACATE_DIANA_35
AGUACATE_DIANA_36
AGUACATE_DIANA_37
AGUACATE_DIANA_38
AGUACATE_DIANA_39
AGUACATE_DIANA_40
AGUACATE_DIANA_41
AGUACATE_DIANA_42
AGUACATE_DIANA_43
AGUACATE_DIANA_44
AGUACATE_DIANA_45
AGUACATE_DIANA_46
AGUACATE_DIANA_47
AGUACATE_DIANA_48
AGUACATE_DIANA_49
AGUACATE_DIANA_50
AGUACATE_DIANA_51
AGUACATE_DIANA_52
AGUACATE_DIANA_53
AGUACATE_DIANA_54
AGUACATE_DIANA_55
AG

In [11]:
region = 'AGUACATE_DIANA_0'
region_geom = Region.objects.get(name=region).the_geom
region_geom

In [12]:
region_geom.wkt

'POLYGON ((-101.4870389981221 19.27097015907334, -101.4870394335253 19.27095597128261, -101.4870406502424 19.27094182457534, -101.4870426449382 19.27092775772664, -101.4870454121452 19.27091380929275, -101.4870489442785 19.27090001750532, -101.4870532316566 19.27088642016662, -101.4870582625279 19.27087305454596, -101.4870640231031 19.27085995727751, -101.4870704975925 19.27084716425991, -101.4870776682501 19.27083471055785, -101.4870855154213 19.27082263030599, -101.4870940175976 19.27081095661536, -101.4871031514751 19.27079972148265, -101.4871128920182 19.27078895570249, -101.487123212529 19.27077868878304, -101.4871340847195 19.27076894886511, -101.48714547879 19.27075976264506, -101.4871573635099 19.27075115530158, -101.4871697063043 19.2707431504267, -101.4871824733424 19.27073576996116, -101.4871956296308 19.2707290341342, -101.4872091391092 19.2707229614082, -101.4872229647491 19.270717568428, -101.4872370686555 19.27071286997531, -101.4872514121708 19.27070887892823, -101.4872